In [1]:
import torch
import pickle
#import matplotlib as plt
from tdnn import *
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torch.utils.data as Data
import os
import time
import sys
import kaldiio
# from Mydataset import MyDataset
print(sys.executable)




/Users/tanikin/anaconda3/bin/python


In [223]:
f= open("train_label.pkl","rb")

AttributeError: '_io.BufferedReader' object has no attribute 'item'

In [224]:
p= pickle.load(f)

In [225]:
p

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# f.close()

In [279]:
from tqdm import tqdm
class MyDataset(Data.Dataset):
        def __init__(self,filepath,num_path):
#             self.data = []
            self.file = open(filepath,"rb")
            with open(num_path,'rb') as f:
                self.num = pickle.load(f)
        def __len__(self):
            return self.num
        def __getitem__(self,index):
            data,label =pickle.load(self.file)
#             label = pickle.load(self.label)
#             data = self.fopen.__next__()# 自定义transform()对训练数据进行预处理
# #           data = list(zip(x,y))#transform(line)
            return data,label

In [253]:
# with open('train.pkl','rb') as f:
#     X,y = pickle.load(f)



1680

In [257]:

a = -float('inf')#torch.log(torch.tensor(0,dtype= torch.float))
a

-inf

In [284]:
dataset = MyDataset(filepath = 'train_.pkl',num_path = 'utt_num.pkl')

In [285]:

trainloader = Data.DataLoader(dataset, shuffle=False, batch_size=10)

In [288]:
for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    x,y= data
    #inputs, labels = inputs.cuda(), labels.cuda()
    if i == 2:
         break
    #output= net(inputs)
#     print(x)
np.shape(y)

torch.Size([10, 168])

In [275]:
dataset.data.close()
dataset.label.close()

In [33]:
 q= torch.tensor(X,dtype =torch.float,requires_grad=True)

In [8]:
path = os.getcwd()

In [9]:
# p = inputs.argmax(dim=2)
# p.shape
path

'/Users/tanikin/projct'

In [10]:
q.shape

torch.Size([1680, 756, 24])

In [11]:
len(y[0])

168

In [12]:
# mean = q.mean(1)
# std = q.std(1)
# mean.shape
# std.shape
# wexs= torch.cat((mean,std),dim=1)

In [13]:
#mean
#wexs.shape
# qq =q.unsqueeze(1)

# x= F.unfold(qq,(5, 24),stride=(1,24), dilation=(1,1))
# qq.shape
# x = x.transpose(1,2)
# TDNN.kernel(x)

In [14]:
#xxx =nn.Linear(24*5, 512)

In [15]:
len(y[0])

168

In [39]:
net = nn.Sequential()
net.add_module('frame1', TDNN(input_dim=24, output_dim=512, context_size=5, dilation=1))#layer 1
net.add_module('frame2', TDNN(input_dim=512, output_dim=512, context_size=3, dilation=2))#layer 2
net.add_module('frame3', TDNN(input_dim=512, output_dim=512, context_size=3, dilation=3))#layer 3
net.add_module('frame4', TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1))#layer 4
net.add_module('frame5', TDNN(input_dim=512, output_dim=1500, context_size=1, dilation=1))#layer 5
net.add_module('pool',StatsPooling())#pooling
net.add_module('segment6',Segment(input_dim=3000, output_dim=512))#segment1
net.add_module('segment7',Segment(input_dim=512, output_dim=512))#segment2



In [40]:
save_model = torch.load('final_net.pth')

In [41]:
model_dict = net.state_dict()
state_dict = {k:v for k,v in save_model.items() if k in model_dict.keys()}
model_dict.update(state_dict)
net.load_state_dict(model_dict)

<All keys matched successfully>

In [42]:
# for k,v in save_model.items():
#     if k in model_dict.keys():
#         print(k)
net.add_module('softmax',Softmax(input_dim = 512, output_dim = len(y[0])))#softmax
 

In [43]:
#sdsd = net()

#if os.path.exists('trained_net1025.pth'):
        #net.load_state_dict(torch.load('trained_net1025.pth'))
# net

Sequential(
  (frame1): TDNN(
    (kernel): Linear(in_features=120, out_features=512, bias=True)
    (nonlinearity): ReLU()
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame2): TDNN(
    (kernel): Linear(in_features=1536, out_features=512, bias=True)
    (nonlinearity): ReLU()
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame3): TDNN(
    (kernel): Linear(in_features=1536, out_features=512, bias=True)
    (nonlinearity): ReLU()
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame4): TDNN(
    (kernel): Linear(in_features=512, out_features=512, bias=True)
    (nonlinearity): ReLU()
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (frame5): TDNN(
    (kernel): Linear(in_features=512, out_features=1500, bias=True)
    (nonlinearity): ReLU()
    (bn): BatchNorm1d(1500, eps=1e-05, moment

In [44]:
#pp = p.unsqueeze(1)
#pp =F.unfold(X[0],(5,24),stride=1,dilation=1)
inputs = inputs.type(torch.float)
inputs.requires_grad_(True)
out= net(inputs)
# out
labels = labels.type(torch.float)
print(out.argmin(dim=1))
val = out.argmin(dim=1)
ll = labels.argmax(dim = 1)

tensor([409, 258,  75, 140, 354,  38, 257, 364, 451, 426, 457, 300, 329,  89,
         90, 341, 399, 207, 326, 189])


In [45]:
val,ll

(tensor([409, 258,  75, 140, 354,  38, 257, 364, 451, 426, 457, 300, 329,  89,
          90, 341, 399, 207, 326, 189]),
 tensor([460, 460, 460, 460, 460, 460, 460, 460, 460, 460, 461, 461, 461, 461,
         461, 461, 461, 461, 461, 461]))

In [46]:


count = 0
for i in range(len((val))):
    if val[i]==ll[i]:
        count += 1
    
# frame1 = TDNN(input_dim=24, output_dim=512, context_size=5, dilation=1)
# frame2 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=2)
# frame3 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=3)
# frame4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1)
# frame5 = TDNN(input_dim=512, output_dim=1500, context_size=1, dilation=1)
# pool = StatsPooling()
# segment6 = Segment(input_dim=3000, output_dim=512)
# segment7 =Segment(input_dim=512, output_dim=512)
print(count/len(val))

0.0


In [ ]:

# layer1 = frame1(q)# input TDNN(batch, seq_len, input_features)
# layer1

In [ ]:
# layer2 = frame2(layer1)
# layer2


In [ ]:
# layer3 = frame3(layer2)


In [ ]:
# layer4 = frame4(layer3)


In [ ]:
# layer5 = frame5(layer4)


In [ ]:
# layer5

In [ ]:

# 
# layer6 = pool(layer5)


In [ ]:
# layer6

In [ ]:
# se = segment6(layer6)
# se2 = segment7(se)

In [ ]:
# se = segment6(layer6)
# se2 = segment7(se)

In [ ]:
#  sdsd = net(q)
# # sdsd
# layer5.shape
# # layer6.shape
# #mean = layer5.mean(dim=1)


In [ ]:
# q1= torch.tensor(X[5:10],dtype =torch.float,requires_grad=True)
# target= torch.tensor(y[0:5],dtype= torch.float)

In [ ]:
# target1= torch.tensor(y[5:10],dtype= torch.float)
# target

In [ ]:

# sdsd = net(q)
# loss = nn.MSELoss(reduction='sum')

# l = (loss(sdsd,target))
# #output
# optimizer = optim.Adam(net.parameters(), lr=0.02)
# net.zero_grad()
# l.backward()
# optimizer.step()
# print(l)

# # for param in net.parameters():
# #     print(param[0])

# print('=================================')    

# out = net(inputs)

In [ ]:
# for i in range(100):
#     sdsd1 = net(q1)
#     loss = nn.MSELoss(reduction='sum')

#     l = (loss(sdsd1,target1))
#     #output
#     optimizer = optim.Adam(net.parameters(), lr=0.02)
#     net.zero_grad()
#     l.backward()
#     optimizer.step()
#     print('loss is %.4f'%(l))

# # for param in net.parameters():
# #     print(param[0])


In [292]:
num_epoch = 5
for epoch in range(1,num_epoch+1):
    dataset = MyDataset(filepath = 'train_.pkl',num_path = 'utt_num.pkl')
    trainloader = Data.DataLoader(dataset, shuffle=False, batch_size=10)
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        input_, label = data
        #inputs, labels = inputs.cuda(), labels.cuda()
        #output= net(inputs).cuda()
        inputs = input_.type(torch.float)
        inputs.requires_grad_(True)
        labels = label.type(torch.float)
        
        
    dataset.file.close()
#         output = net(inputs)
#         loss = nn.MSELoss(reduction='sum')
        #l = (torch.sum(output*labels))
#         a = torch.max(output)
#         optimizer = optim.Adam(net.parameters(), lr=0.02)
#         net.zero_grad()
#         l.backward()
#         optimizer.step()    
#     print('epoch %d, loss: %f' % (epoch, l.item()))
#     if (epoch % 50) ==0:
#         torch.save(net.state_dict(),'trained_net.pth')
#         print('model saved, epoch: %d'%(epoch))
        

In [8]:
with open('final.raw','rb') as f:
      kaldiweights = pickle.load(f)
#torch.load('final.raw')

UnpicklingError: invalid load key, '\x00'.